**Mochi Toxic Comments Classification Codes**</br>
last update: Dec/08 7:21 AM</br>
</br>
This notebook contains code for group Mochi's project</br>
Following codes are included</br>
0. set up hyperparameters</br>
1. Prepossessing Data</br>
 1. extract data from dataset</br>
 2. divide into training, testing, validation datasets</br>
 3. do weighting (if weighting = True)</br>
 4. save dataLoader into file</br>
2. Define Model and Train Model</br>
 1. load dataLoader from file</br>
 2. define model</br>
 3. train model</br>
3. Evaluate Model</br>
 1. evaluate model using testing dataset</br>
 2. save hyparameters and evaluations into file</br>

note:</br>
1. multi labeling input -> [0,0,1,1,0]
2. multi labeling output ->[0.001, 0.01, 0.6, 0.7, 0.0001]

In [ ]:
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"
from google.colab import drive
drive.mount('/content/drive')
import csv
import torch
from torch import nn
import numpy as np
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator
import torch.nn.functional as F
import tensorflow as tf
from torch.utils import data
from tqdm import tqdm
import random
import pickle
torch.cuda.is_available()

Mounted at /content/drive


True

In [ ]:
#https://www.kaggle.com/c/jigsaw-unintended-bias-in-toxicity-classification/data
#Download data from the above url, and upload it in the google drive "Mochi" folder
#test = 182375
#train = 1684759
#used only approved data[7], and threshold is set to 0.5 with category "toxicity"[13]
BASE_DIR = '/content/drive/My Drive/Mochi'
src_file = os.path.join(BASE_DIR,"all_data.csv")
temp = False # True for test(small size) False for real(big size)

In [ ]:
#https://www.kaggle.com/c/jigsaw-unintended-bias-in-toxicity-classification/data
#Download data from the above url, and upload it in the google drive "Mochi" folder
#test = 182375
#train = 1684759
#used only approved data[7], and threshold is set to 0.5 with category "toxicity"[13]
#returns train/test/val_toxic/type/target each array represents classes(toxic/non-toxic, types of toxic, targets of toxic)
#more detail about the classes are below
def csv_processing(src_file, temp):
  tox = 0
  norm = 0
  test_set = True

  f = open(src_file, 'r', encoding='utf-8')
  src = csv.reader(f)
  train = []
  train_toxic = []
  train_type = []
  train_target = []
  test = []
  test_toxic = []
  test_type = []
  test_target = []
  val = []
  val_toxic = []
  val_type = []
  val_target = []
  types = []
  targets = []
  if temp == True:
    rate = 0.1#(10% of train data)
  else:
    rate = 0.01#(1% of train data)
  for count, line in enumerate(src):
    if count == 0:
      types = types + line[14:20]
      targets = targets + line[20:44]
      continue
    if float(line[13]) == 0.0 and temp == True: ##temp for reducing the size of train set
      break
    if line[7] == 'approved':
      if line[2] == 'train':
        if random.random()>rate: #0.1 for temp, 0.01 for real
          train.append(line[1])
          binary, toxic_type, target = classification(line)
          train_toxic.append(binary)
          train_type.append(toxic_type)
          train_target.append(target)
        else:
          val.append(line[1])
          binary, toxic_type, target = classification(line)
          val_toxic.append(binary)
          val_type.append(toxic_type)
          val_target.append(target)
      if line[2] == 'test' and test_set == True:
        if float(line[13]) >= 0.5:
          tox = tox +1
        else:
          norm = norm +1
        if float(line[13]) == 0.0 and tox-norm == 0:
          test_set = False
        test.append(line[1])
        binary, toxic_type, target = classification(line)
        test_toxic.append(binary)
        test_type.append(toxic_type)
        test_target.append(target)
    
  train_toxic = torch.tensor(train_toxic)
  test_toxic = torch.tensor(test_toxic)
  val_toxic = torch.tensor(val_toxic)
  train_type = torch.tensor(train_type)
  test_type = torch.tensor(test_type)
  val_type = torch.tensor(val_type)
  train_target = torch.tensor(train_target)
  test_target = torch.tensor(test_target)
  val_target = torch.tensor(val_target)
  f.close()

  return train, train_toxic, train_type, train_target, test, test_toxic, test_type, test_target, val, val_toxic, val_type, val_target, types, targets

def classification(line):
  toxic_type = [0,0,0,0,0,0]
  target = [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]
  if float(line[13]) >= 0.5:
    binary = 1
    type_value = max(line[14:20])
    for i in range(14,20):
      if float(line[i]) >= 0.5:
        toxic_type[i-14] = 1
      else:
        toxic_type[i-14] = 0
    if line[20] != '':
      for i in range(20,44):
        if float(line[i]) >= 0.5:
          target[i-20] = 1
        else:
          target[i-20] = 0
  else:
    binary = 0

  return binary, toxic_type, target

In [ ]:
train, train_toxic, train_type, train_target, test, test_toxic, test_type, test_target, val, val_toxic, val_type, val_target, types, targets = csv_processing(src_file, temp)

In [ ]:
#more details about the classes to be classified
print(len(train))
print(len(test))
print(len(val))
print(types) # each index represents the class of train/test/val_type
print(targets)# each index represents the class of train/test/val_targets
print(len(types))
print(len(targets))

1667834
1098
16925
['severe_toxicity', 'obscene', 'sexual_explicit', 'identity_attack', 'insult', 'threat']
['male', 'female', 'transgender', 'other_gender', 'heterosexual', 'homosexual_gay_or_lesbian', 'bisexual', 'other_sexual_orientation', 'christian', 'jewish', 'muslim', 'hindu', 'buddhist', 'atheist', 'other_religion', 'black', 'white', 'asian', 'latino', 'other_race_or_ethnicity', 'physical_disability', 'intellectual_or_learning_disability', 'psychiatric_or_mental_illness', 'other_disability']
6
24


In [ ]:
special_chars = ['`','~','!','@','#','^','(',')','[','{','}',']',';',':','*','$','%','&','-','_','=','+','\'','"','\\','\n','\t','|',',','<','.','>','/','?']
def sp_char_processing(data_set, special_chars):
  for i in range(len(data_set)):
    text = data_set[i]
    for char in special_chars:
      text = text.replace(char,'')
    data_set[i] = "<sos> " + text + " <eos>"

  return data_set

In [ ]:
#https://pytorch.org/tutorials/beginner/text_sentiment_ngrams_tutorial.html
#constructing our own vocabulary
train = sp_char_processing(train, special_chars)
test = sp_char_processing(test, special_chars)
val = sp_char_processing(val, special_chars)
tokenizer = get_tokenizer('basic_english')

def yield_tokens(data_iter):
    for text in data_iter:
        yield tokenizer(text)

vocab = build_vocab_from_iterator(yield_tokens(train), specials=["<unk>",'<pad>','<sos>','<eos>']) #0, 1, 2, 3 ,min_freq = 1
vocab.set_default_index(vocab["<unk>"])

In [ ]:
# visualizing the dataset
print(train[5000])
print(tokenizer(train[5000]))
print(vocab(tokenizer(train[5000])))
print(test[1])
print(vocab(tokenizer(test[1])))

<sos> It is the same idiots who reduced the Old Age Security pension qualifying age from 67 to 65 yearsCanadians elected a self destructive Government believing in heart outward economy and self balancing budget headed by a juvenile drama king  They are getting what they deserved <eos>
['<sos>', 'it', 'is', 'the', 'same', 'idiots', 'who', 'reduced', 'the', 'old', 'age', 'security', 'pension', 'qualifying', 'age', 'from', '67', 'to', '65', 'yearscanadians', 'elected', 'a', 'self', 'destructive', 'government', 'believing', 'in', 'heart', 'outward', 'economy', 'and', 'self', 'balancing', 'budget', 'headed', 'by', 'a', 'juvenile', 'drama', 'king', 'they', 'are', 'getting', 'what', 'they', 'deserved', '<eos>']
[2, 14, 9, 4, 118, 2558, 38, 1880, 4, 251, 681, 609, 1537, 12262, 681, 36, 6238, 5, 3208, 287344, 497, 8, 912, 4089, 103, 3112, 10, 1109, 15187, 427, 6, 912, 7919, 504, 3380, 35, 8, 5355, 3494, 1673, 23, 16, 265, 34, 23, 4496, 3]
<sos> NO   There are no alternative facts Go check for 

In [ ]:
##############################
######hyperparameters#########
##############################
classification = 'binary' #one of 'binary', 'type', 'target'
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
embed_dim = 256
max_len = 100 #train avg 61, test avg 55
batch_size = 512
hidden_size = 500
learning_rate = 0.001
num_layer = 1
bidirectional = True
num_epochs = 4
if classification == 'binary':
  num_classes = 1 # 1 for binary, 6 for types, 24 for targets
elif classification == 'type':
  num_classes = 6
elif classification == 'target':
  num_classes = 24
use_GLoVe = False

In [ ]:
#tokenizing + indexing + maxlen filtering
# and padding
#
# After,  tokened is sentence in word tokens (each sentence has 100 word tokens)
#         bin_set is sentence labels (toxic or not) ([1,0] for toxic, [0,1] for not toxic) (binary type targets)
#         data_set is a list of sentences (in words)
def tokenizing(data_set, class_set, classification):
  tokened = torch.zeros(len(data_set),max_len, dtype=int)
  class_sett = class_set.clone()
  original = []
  count = 0
  for i in range(len(data_set)):
    if i % int(len(data_set)*0.1) == 0:
      print(str(i) + "/" + str(len(data_set)))
    token = tokenizer(data_set[i])
    if len(token) > max_len:
      continue
    if (classification == "type" or classification == "target") and sum(class_set[i]) == 0:
      continue
    token = torch.tensor(vocab(token), dtype=int)
    token = F.pad(token, pad=(0, max_len - len(token)), mode='constant', value=vocab(['<pad>'])[0])
    tokened[count] = token
    class_sett[count] = class_set[i]
    original.append(data_set[i])
    count = count + 1
  print("done")
  print(count)
  tokened = tokened[:count,:]
  class_sett = class_sett[:count]
  data_set = original
  return tokened, class_sett, data_set

In [ ]:
# the direct input to the RNN model
# constructing the loader of the data sets
if classification == "binary":
  train_set = train_toxic
  test_set = test_toxic
  val_set = val_toxic
elif classification == "type":
  train_set = train_type
  test_set = test_type
  val_set = val_type
elif classification == "target":
  train_set = train_target
  test_set = test_target
  val_set = val_target

train_tokened, train_sett, _ = tokenizing(train, train_set, classification)
train_dataset = data.TensorDataset(train_tokened, train_sett)
train_loader = data.DataLoader(dataset = train_dataset, batch_size = batch_size, shuffle = True)

test_tokened, test_sett, _ = tokenizing(test, test_set, classification)
test_dataset = data.TensorDataset(test_tokened, test_sett)
test_loader = data.DataLoader(dataset = test_dataset, batch_size = 1, shuffle = True)

val_tokened, val_sett, _ = tokenizing(val, val_set, classification)
val_dataset = data.TensorDataset(val_tokened, val_sett)
val_loader = data.DataLoader(dataset = val_dataset, batch_size = int(batch_size/2), shuffle = True)

0/1667688
166768/1667688
333536/1667688
500304/1667688
667072/1667688
833840/1667688
1000608/1667688
1167376/1667688
1334144/1667688
1500912/1667688
1667680/1667688
done
1401455
0/1098
109/1098
218/1098
327/1098
436/1098
545/1098
654/1098
763/1098
872/1098
981/1098
1090/1098
done
997
0/17071
1707/17071
3414/17071
5121/17071
6828/17071
8535/17071
10242/17071
11949/17071
13656/17071
15363/17071
17070/17071
done
14295


In [ ]:
# saving dataLoaders into files, so dont need to tokenlize data everytime.
if classification == "binary":
  data_loader_dir = os.path.join(BASE_DIR,"BinarydataLoader")
elif classification == "type":
  data_loader_dir = os.path.join(BASE_DIR,"TypedataLoader")
elif classification == "target":
  data_loader_dir = os.path.join(BASE_DIR,"TargetdataLoader")

if(not os.path.isdir(data_loader_dir)):
  os.mkdir(data_loader_dir)

torch.save(train_loader, os.path.join(data_loader_dir,"train_loader.loader"))
torch.save(test_loader, os.path.join(data_loader_dir,"test_loader.loader"))
torch.save(val_loader, os.path.join(data_loader_dir,"val_loader.loader"))

In [ ]:
#https://nlp.stanford.edu/projects/glove/
# here is GLoVe embedding
# implementation follows https://medium.com/@martinpella/how-to-use-pre-trained-word-embeddings-in-pytorch-71ca59249f76
# about 80% words in dataset is not in Glove, not using it
'''
words = []
idx = 0
word2idx = {}
vectors = []

with open(os.path.join(BASE_DIR,"glove.42B.300d.txt"), 'rb') as f:
    for l in f:
        line = l.decode().split()
        word = line[0]
        words.append(word)
        word2idx[word] = idx
        idx += 1
        vect = np.array(line[1:]).astype(np.float)
        vectors.append(vect)

glove = {}
found_words = 0
unfound_words = 0
vocab_stoi = vocab.get_stoi()
for w in vocab_stoi:
  if w in word2idx:
    found_words += 1
    glove[vocab_stoi[w]] = vectors[word2idx[w]]
  else:
    unfound_words += 1
    glove[vocab_stoi[w]] = np.random.normal(scale=0.6, size=100)

print("found words:", found_words)
print("unfound words", unfound_words)

file = open(os.path.join(BASE_DIR,"glove.42B.300d.embedding_dict"), "wb")
pickle.dump(glove, file)
file.close()
'''

'\nwords = []\nidx = 0\nword2idx = {}\nvectors = []\n\nwith open(os.path.join(BASE_DIR,"glove.42B.300d.txt"), \'rb\') as f:\n    for l in f:\n        line = l.decode().split()\n        word = line[0]\n        words.append(word)\n        word2idx[word] = idx\n        idx += 1\n        vect = np.array(line[1:]).astype(np.float)\n        vectors.append(vect)\n\nglove = {}\nfound_words = 0\nunfound_words = 0\nvocab_stoi = vocab.get_stoi()\nfor w in vocab_stoi:\n  if w in word2idx:\n    found_words += 1\n    glove[vocab_stoi[w]] = vectors[word2idx[w]]\n  else:\n    unfound_words += 1\n    glove[vocab_stoi[w]] = np.random.normal(scale=0.6, size=100)\n\nprint("found words:", found_words)\nprint("unfound words", unfound_words)\n\nfile = open(os.path.join(BASE_DIR,"glove.42B.300d.embedding_dict"), "wb")\npickle.dump(glove, file)\nfile.close()\n'

In [ ]:
# load dataLoaders from file
if classification == "binary":
  data_loader_dir = os.path.join(BASE_DIR,"BinarydataLoader")
elif classification == "type":
  data_loader_dir = os.path.join(BASE_DIR,"TypedataLoader")
elif classification == "target":
  data_loader_dir = os.path.join(BASE_DIR,"TargetdataLoader")

train_loader = torch.load(os.path.join(data_loader_dir,"train_loader.loader"))
test_loader = torch.load(os.path.join(data_loader_dir,"test_loader.loader"))
val_loader = torch.load(os.path.join(data_loader_dir,"val_loader.loader"))
print(data_loader_dir)

/content/drive/My Drive/Mochi/TypedataLoader


In [ ]:
#distribution and weight calculation
if classification == 'type':
  indexes = [0,0,0,0,0,0]
  for (_,labels) in train_loader:
    for label in labels:
      for i in range(num_classes):
        if label[i] != 0:
          indexes[i] += 1
  print(indexes)
elif classification == 'target':
  indexes = [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]
  for (_,labels) in train_loader:
    for label in labels:
      for i in range(num_classes):
        if label[i] != 0:
          indexes[i] += 1
  print(indexes)

#for i in range(num_classes):
#  indexes[i] = 1/indexes[i]
#weights = indexes # for unbalanced data.(larger classes have less weights)). This is for criterion parameter in training code sectiion
#weights[0] = 0.0
#class_weights = torch.FloatTensor(weights).cuda()

In [ ]:
#Evaluation Functions
def accuracy(loader):
  correct = 0
  incorrect = 0
  count = 0
  for (input, label) in loader:
    if count%int(len(loader)*0.1) == 0:
      print(str(count) + "/" + str(len(loader)))
    count = count + 1
    input = input.to(device)
    pred = model(input)
    if(num_classes==1):
      if pred>=0.5:
        res = 1
      else:
        res = 0
      if res == label:
        correct = correct + 1
      else:
        incorrect = incorrect + 1
    elif(num_classes>1):
      if(label[0,torch.argmax(pred).item()].tolist()):
        correct = correct + 1
      else:
        incorrect = incorrect + 1
  print(float(correct/(correct+incorrect)))
  print(correct)
  print(incorrect)
  return (correct, incorrect)

# following the evaluation metrics in https://en.wikipedia.org/wiki/Multi-label_classification
def F1ScoreCalculation(loader, model):
  threshold = 0.5
  T = 0       # size of set of truth labels
  P = 0       # size of set of predicated labels
  T_u_P = 0   # size of T union P
  T_n_P = 0   # size of T intersection P
  count = 0   # for iterating
  for (input, label) in loader:
    if count%int(len(loader)*0.1) == 0:
      print(str(count) + "/" + str(len(loader)))
    count = count + 1
    input = input.to(device)
    pred = model(input)

    pred_list = pred.tolist()
    label_list = label.tolist()
    for i in range(num_classes):
      if num_classes > 1:
        if(pred_list[0][i] > threshold):
          P += 1
        if(label_list[0][i] > threshold):
          T += 1
        if(pred_list[0][i] > threshold and label_list[0][i] > threshold):
          T_n_P += 1
      elif num_classes == 1:
        if(pred_list[0][i] > threshold):
          P += 1
        if(label_list[0] > threshold):
          T += 1
        if(pred_list[0][i] > threshold and label_list[0] > threshold):
          T_n_P += 1
  T_u_P = T + P - T_n_P

  print("total predicted label num:", P)
  print("total truth label num:", T)

  if(T_n_P == 0):
    return 0
  assert P!=0, "predicted label number is 0"
  assert T!=0, "truth label number is 0"

  precision = T_n_P/P
  recall = T_n_P/T
  F1 = 2/(1/precision + 1/recall)
  print("F1:",F1)
  print("precision:",precision)
  print("recall:",recall)
  return (precision, recall, F1)



In [ ]:
#if classification == "type" or classification == "target":
#  count = 0
#  for (input, label) in test_loader:
#    count += 1
#    input = input.to(device)
#    pred = model(input)
#    print(input)
#    print(pred)
#    print(label)
#    print(int(torch.argmax(pred).item()))
#    print(label[0,4].tolist())
#    if(count==10):
#      break

In [ ]:
print(len(vocab))
print(vocab(['<pad>'])[0])

1034069
1


In [ ]:
# define the RNN model 
class RNN(nn.Module):
  def __init__(self, input_size, hidden_size, num_layers, num_classes):
    super(RNN, self).__init__()
    self.num_layers = num_layers
    self.hidden_size = hidden_size
    self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True, bidirectional=bidirectional)#, dropout = 0.2) #bidirectional
    if bidirectional:
      self.fc = nn.Linear(hidden_size*2, num_classes)
    else:
      self.fc = nn.Linear(hidden_size, num_classes)
    self.word_embedding = nn.Embedding(len(vocab), embed_dim, padding_idx=vocab(['<pad>'])[0])
    #self.word_embedding = nn.Embedding(1034266, embed_dim, padding_idx=1)  #calculated from default vocab this mag change over time (?)
    self.activation = nn.Sigmoid()

  def forward(self, x):
    out = self.word_embedding(x).to(device)
    if bidirectional:
      h0 = torch.autograd.Variable(torch.ones(self.num_layers*2, x.size(0), self.hidden_size)).to(device)
      c0 = torch.autograd.Variable(torch.ones(self.num_layers*2, x.size(0), self.hidden_size).to(device))
    else:
      h0 = torch.autograd.Variable(torch.ones(self.num_layers, x.size(0), self.hidden_size)).to(device)
      c0 = torch.autograd.Variable(torch.ones(self.num_layers, x.size(0), self.hidden_size).to(device))
    out,(h0,c0) = self.lstm(out, (h0,c0))
    out = out[:, -1, :]
    out = self.fc(out)
    out = self.activation(out)
    return out

model = RNN(embed_dim, hidden_size, num_layer, num_classes).to(device)   # num_classes=2 because we are doing binary

In [ ]:
F1ScoreCalculation(test_loader, model) #F1 score before train

0/480
48/480
96/480
144/480
192/480
240/480
288/480
336/480
384/480
432/480
total predicted label num: 486
total truth label num: 530
F1: 0.003937007874015748
precision: 0.00411522633744856
recall: 0.0037735849056603774


0.003937007874015748

In [ ]:
# Train the RNN model
#criterion = nn.CrossEntropyLoss()
criterion = nn.BCELoss() #weight=class_weights
  optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate) 

n_total_steps = len(train_loader)
for epoch in range(num_epochs):
    for i, (inputs, labels) in enumerate(train_loader):
        # shape of "inputs" is [batch_size, max_len]
        # shape of "labels" is [batch_size, 1]

        inputs = inputs.to(device)
        #labels = labels.reshape(-1).to(device)
        labels = labels.to(device, dtype=torch.float)
        labels = labels.reshape(labels.shape[0],num_classes)
        # Forward pass
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if (i+1) % int(len(train_loader)/4) == 0:
            count = 0
            loss_total = 0
            for (val_input, val_label) in val_loader: #validation loss calculation to see the progress of the training
              val_input = val_input.to(device)
              val_label = val_label.to(device, dtype=torch.float)
              val_label = val_label.reshape(val_label.shape[0],num_classes)
              val_output = model(val_input)
              loss_total = loss_total + criterion(val_output, val_label).item()
              count = count + 1
            print (f'Epoch [{epoch+1}/{num_epochs}], Step [{i+1}/{n_total_steps}], Train Loss: {loss.item():.10f}, Val Loss: {loss_total/count:.10f}')

Epoch [1/4], Step [684/2738], Train Loss: 0.2864159644, Val Loss: 0.2525187843
Epoch [1/4], Step [1368/2738], Train Loss: 0.2370384037, Val Loss: 0.2492833805
Epoch [1/4], Step [2052/2738], Train Loss: 0.1504172981, Val Loss: 0.1457506098
Epoch [1/4], Step [2736/2738], Train Loss: 0.1466296166, Val Loss: 0.1361152676
Epoch [2/4], Step [684/2738], Train Loss: 0.0826305971, Val Loss: 0.1327181523
Epoch [2/4], Step [1368/2738], Train Loss: 0.1053089648, Val Loss: 0.1264442786
Epoch [2/4], Step [2052/2738], Train Loss: 0.1232839301, Val Loss: 0.1213350939
Epoch [2/4], Step [2736/2738], Train Loss: 0.1109443605, Val Loss: 0.1212815364
Epoch [3/4], Step [684/2738], Train Loss: 0.0970210135, Val Loss: 0.1231976793
Epoch [3/4], Step [1368/2738], Train Loss: 0.1335400939, Val Loss: 0.1257446721
Epoch [3/4], Step [2052/2738], Train Loss: 0.1095059291, Val Loss: 0.1237182116
Epoch [3/4], Step [2736/2738], Train Loss: 0.1136402041, Val Loss: 0.1215718448
Epoch [4/4], Step [684/2738], Train Loss: 0

In [ ]:
F1 = F1ScoreCalculation(test_loader, model) #accuracy after train

notefile = '/content/drive/My Drive/Mochi/_multi_class_notes.txt'
log = "\n"
if use_GLoVe:
  log += "embed_dim: " + str(embed_dim) + "(in GLoVe)\n"
else:
  log += "embed_dim: " + str(embed_dim) + "\n"
log += "max_len: " + str(max_len) + "\n"
log += "batch_size: " + str(batch_size) + "\n"
log += "hidden_size: " + str(hidden_size) + "\n"
log += "learning_rate: " + str(learning_rate) + "\n"
log += "num_layer: " + str(num_layer) + "\n"
log += "num_class: " + str(num_classes) + "\n"
log += "num_epochs: " + str(num_epochs) + "\n"
log += "precision: " + str(F1[0]) + "\n"
log += "recall: " + str(F1[1]) + "\n"
log += "F1: " + str(F1[2]) + "\n"
if use_GLoVe:
  log += "using GLove embedding \n"
if bidirectional:
  log += "using bidirectional LSTM \n"
myfile = open(notefile, 'a', encoding='utf-8')
myfile.write(log)
myfile.close()

0/997
99/997
198/997
297/997
396/997
495/997
594/997
693/997
792/997
891/997
990/997
total predicted label num: 421
total truth label num: 509
F1: 0.8430107526881722
precision: 0.9311163895486936
recall: 0.7701375245579568
